In [1]:
%matplotlib qt

import numpy as np
from matplotlib.pyplot import *
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
import sympy as sp

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

In [11]:
def lagrange_polynomial(x, points):
    n = len(points)
    expr = sp.S(0)
    
    for i, ipoint in enumerate(points):
        numerator = sp.S(1)
        for j, jpoint in enumerate(points):
            if i != j:
                numerator *= x - jpoint[0]
                
        denominator = numerator.subs(x, ipoint[0])
        expr += numerator / denominator * ipoint[1]
    
    return sp.expand(expr)


t = sp.symbols('t')
points = [(0, 0),
          (1, 1),
          (2, 0),
          (3, 0),
          (4, 0),
          (5, 0),
          (6, 0)]
poly = lagrange_polynomial(t, points)
figure()
min_t = min([point[0] for point in points])
max_t = max([point[0] for point in points])
ts = np.arange(min_t, max_t+0.05, 0.05)
plot(ts, [sp.N(poly.subs(t, t_val)) for t_val in ts])
grid()
show()

In [47]:
def trajectory():
    t = sp.symbols('t')
    x = sp.sin(sp.pi * t)
    y = sp.cos(sp.pi * t)
    z = - t
    psi = - 2*sp.pi*t
    theta = 0
    phi = - sp.pi / 3
    return t, x, y, z, phi, theta, psi

def gen_traj_funcs(t, x, y, z, phi, theta, psi):
    xp = sp.lambdify(t, x)
    yp = sp.lambdify(t, y)
    zp = sp.lambdify(t, z)
    vx = sp.diff(x, t)
    vy = sp.diff(y, t)
    vz = sp.diff(z, t)
    ax = sp.diff(vx, t)
    ay = sp.diff(vy, t)
    az = sp.diff(vz, t)
    wx = sp.diff(phi, t) - sp.diff(psi, t)*sp.sin(theta)
    wy = sp.diff(theta, t)*sp.cos(phi) + sp.diff(psi, t)*sp.cos(theta)*sp.sin(phi)
    wz = -sp.diff(theta, t)*sp.sin(phi) + sp.diff(psi, t)*sp.cos(theta)*sp.cos(phi)
    
    def c(x_):
        return sp.cos(x_)
        
    def s(x_):
        return sp.sin(x_)
    
    R = sp.Matrix([[c(theta)*c(psi), c(theta)*s(psi), -s(theta)],
         [s(phi)*s(theta)*c(psi)-c(phi)*s(psi), s(phi)*s(theta)*s(psi)+c(phi)*c(psi), s(phi)*c(theta)],
         [c(phi)*s(theta)*c(psi)+s(phi)*s(psi), c(phi)*s(theta)*s(psi)-s(phi)*c(psi), c(phi)*c(theta)]])
    
    f = R*(sp.Matrix([[ax, ay, az]]).T + sp.Matrix([[0, 0, 9.81]]).T)
    
    phip = sp.lambdify(t, phi)
    thetap = sp.lambdify(t, theta)
    psip = sp.lambdify(t, psi)
    
    return xp, yp, zp, phip, thetap, psip, sp.lambdify(t, wx), sp.lambdify(t, wy), sp.lambdify(t, wz), sp.lambdify(t, f[0]), sp.lambdify(t, f[1]), sp.lambdify(t, f[2])


t, x, y, z, psi, theta, phi = trajectory()
x, y, z, phi, theta, psi, wx, wy, wz, fx, fy, fz = gen_traj_funcs(t, x, y, z, psi, theta, phi)

In [48]:
fig = figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_aspect('equal')

ts = np.arange(0, 5, 0.05)
ax.plot([x(t) for t in ts], [y(t) for t in ts], [z(t) for t in ts])
arrow = Arrow3D([0, 0.5], [0, 0.5], [1, 1.5], lw=3, mutation_scale=20, arrowstyle="-|>", color="red")
ax.add_artist(arrow)

show()

fig2 = figure()
plot(ts, [fx(t) for t in ts])
plot(ts, [fy(t) for t in ts])
plot(ts, [fz(t) for t in ts])